**Preamble**

- This code cleans the raw public procurement data obtained from the State Audit Courts (TCEs) of the following states: CE, PE, MG, PR, RS and PB.
- The final output of this code is the tender participant table (_licitacao participante_), available at [basedosdados](https://basedosdados.org/dataset/d3874769-bcbd-4ece-a38a-157ba1021514?table=14c5d05b-9830-4710-b7ac-7e0ca1bf9d8b).
- Made by: Nathalia Sales


In [ ]:
import glob
import os
from zipfile import ZipFile

import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
# Connect to google drive

drive.mount("/content/gdrive")

# Display options

pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:.2f}".format

# Set directory

path = "/content/gdrive/MyDrive/ComprasPublicas_Brasil"

# Open some auxiliary files

municipio = pd.read_csv(
    os.path.join(path, "auxiliary_files/municipio.csv"),
    encoding="utf-8",
    dtype=str,
)

id_tce = pd.read_csv(
    os.path.join(path, "input/PE/municipios.csv"),
    encoding="latin-1",
    dtype=str,
    usecols=["CODIGOIBGE", "CODIGO", "UNIDADEFEDERATIVA"],
)

id_tce = id_tce.rename(
    columns={
        "CODIGOIBGE": "id_municipio",
        "CODIGO": "id_municipio_tce",
        "UNIDADEFEDERATIVA": "sigla_uf",
    },
)

# Merge both
municipio = municipio.merge(
    id_tce,
    how="left",
    left_on=["id_municipio", "sigla_uf"],
    right_on=["id_municipio", "sigla_uf"],
)

ug_id = pd.read_csv(
    os.path.join(path, "auxiliary_files/ug_id_mg.csv"), sep=",", dtype=str
)  # MG

orgao_municipio = pd.read_csv(
    os.path.join(path, "input/RS/orgaos_auditados_rs.csv"),
    encoding="utf-8",
    dtype=str,
    usecols=["CD_MUNICIPIO_IBGE", "CD_ORGAO"],
)  # RS

# Create a list of UFs
ufs = municipio["sigla_uf"].unique().tolist()

# Set columns order

ordem = [
    "ano",
    "sigla_uf",
    "id_municipio",
    "orgao",
    "id_unidade_gestora",
    "id_licitacao_bd",
    "id_licitacao",
    "id_dispensa",
    "razao_social",
    "documento",
    "habilitado",
    "classificado",
    "vencedor",
    "endereco",
    "cep",
    "municipio_participante",
    "tipo",
]

# CE


In [ ]:
# CE

# Get a list of all CSV files

all_files_ce_licitantes = glob.glob(
    os.path.join(path, "input/CE/Licitações/licitantes_*.csv")
)

# Initialize an empty list and loop through each CSV file

all_df_ce = []
for f in all_files_ce_licitantes:
    df1 = pd.read_csv(f, sep=";", encoding="latin-1", dtype=str)
    df1["arquivo"] = f.split("/")[-1]
    all_df_ce.append(df1)

# Concatenate all DataFrames in the list into a single DataFrame

ce1 = pd.concat(all_df_ce, ignore_index=True, sort=True)

# Extract the year from the file name
ce1["ano"] = ce1["arquivo"].str[11:15]

# Drop and rename
ce1_drop = ["fone_negociante", "arquivo", "codigo_uf "]

ce1_rename = {
    "numero_licitacao": "id_licitacao",
    "codigo_tipo_negociante": "tipo",
    "nome_negociante": "razao_social",
    "endereco_negociante": "endereco",
    "cep_negociante": "cep",
    "nome_municipio_negociante": "municipio_participante",
    "numero_documento_negociante": "documento",
}

ce1 = ce1.drop(ce1_drop, axis=1)
ce1 = ce1.rename(ce1_rename, axis=1)

# Read a CSV file containing municipality information

id_mun = pd.read_csv(
    os.path.join(path, "input/CE/municipios.csv"),
    sep=";",
    dtype=str,
    encoding="latin-1",
    usecols=["geoibgeId", "codigo_municipio"],
)

id_mun = id_mun.rename({"geoibgeId": "id_municipio"}, axis=1)

# Merge on codigo_municipio to get id_municipio (IBGE code)

ce1 = ce1.merge(
    id_mun,
    how="left",
    left_on="codigo_municipio",
    right_on="codigo_municipio",
)

# Assign state acronym to the 'sigla_uf'
ce1["sigla_uf"] = "CE"

# Create a unique identifier for each purchase
ce1["id_licitacao_bd"] = (
    ce1["id_licitacao"]
    + ce1["id_municipio"]
    + ce1["ano"].str[2:4]
    + ce1["sigla_uf"]
)

# Merge with licitacao to deal with duplicates in id_licitacao_bd

ce2 = pd.read_csv(
    os.path.join(path, "output/licitacao_ce.csv"),
    dtype=str,
    encoding="utf-8",
    usecols=["id_municipio", "ano", "id_licitacao", "id_licitacao_bd"],
)

ce2 = ce2.rename({"id_licitacao_bd": "id_licitacao_bd_2"}, axis=1)

ce1 = ce1.merge(
    ce2,
    how="left",
    left_on=["id_municipio", "ano", "id_licitacao"],
    right_on=["id_municipio", "ano", "id_licitacao"],
    indicator=True,
)

# If id_licitacao (which comes from the original variable numero_licitacao), ano and id_municipio are the same
# in both licitacao and participants table, but id_licitacao_bd is different, it must be the case that we cannot
# uniquely identify that tender. In those cases, we set the id as missing.

ce1["id_licitacao_bd"] = np.where(
    (ce1["_merge"] == "both")
    & (ce1["id_licitacao_bd"] != ce1["id_licitacao_bd_2"]),
    np.nan,
    ce1["id_licitacao_bd"],
)

# Drop non-necessary variable
ce1 = ce1.drop("data_realizacao_licitacao", axis=1)

# Apply standard format to documents/cnpjs
ce1["length"] = ce1["documento"].str.len()
ce1["documento"] = np.where(
    ce1["length"] == 13, ce1["documento"].str.zfill(14), ce1["documento"]
)
ce1["documento"] = np.where(
    ce1["length"] == 15, ce1["documento"].str[1:], ce1["documento"]
)

# Merge with items to get winners

ce3 = pd.read_csv(
    os.path.join(path, "output/licitacao_item_ce.csv"),
    encoding="utf-8",
    dtype=str,
    usecols=["id_municipio", "ano", "id_licitacao_bd", "documento"],
)

# One supplier may appear many times in the same tender by winning different items
# Keep it unique

ce3 = ce3.drop_duplicates(subset=["documento", "id_licitacao_bd"])

ce3["vencedor"] = "1"

# Merge suppliers and items
ce = ce1.merge(
    ce3,
    how="left",
    left_on=["id_municipio", "ano", "id_licitacao_bd", "documento"],
    right_on=["id_municipio", "ano", "id_licitacao_bd", "documento"],
)

ce["vencedor"] = ce["vencedor"].replace(np.nan, "0")

# Drop duplicates in all variables
ce = ce.drop_duplicates()

# Six cases where there are small differences in address, keep the last
ce = ce.drop_duplicates(subset=["documento", "id_licitacao_bd"], keep="last")

# Reorder columns
ce = ce.reindex(columns=ordem)

# Save
ce.to_csv(
    os.path.join(path, "output/licitacao_participante_ce.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

# PE


In [ ]:
# PE

# Get a list of all CSV files

all_files = glob.glob(
    os.path.join(path, "input/PE/Licitações/licitacoesdetalhes_*.csv")
)

# Initialize an empty list and loop through each CSV file

all_df = []
for f in all_files:
    df1 = pd.read_csv(f, sep=",", encoding="latin-1", dtype=str)
    df1["arquivo"] = f.split("/")[-1]
    all_df.append(df1)

# Concatenate all DataFrames in the list into a single DataFrame

pe = pd.concat(all_df, ignore_index=True, sort=True)

# Extract the year from the file name

pe["ano"] = pe["arquivo"].str[19:23]

# List original variables to drop

columns_pe = [
    "DESCRICAOOBJETO",
    "DATAPUBLICACAOHOMOLOGACAO",
    "LinkArquivo",
    "CODIGOOBJETO",
    "VALORORCAMENTOESTIMATIVO",
    "CODIGOSITUACAOLICITACAO",
    "UG",
    "ESTAGIOLICITACAO",
    "NOMEMODALIDADE",
    "OBJETOCONFORMEEDITAL",
    "CODIGODESCRICAOOBJETO",
    "QTDELICITANTES",
    "NOMENATUREZA",
    "TOTALADJUDICADOLICITANTE",
    "NUMEROMODALIDADE",
    "CODIGOESTAGIOLICITACAO",
    "TOTALADJUDICADOLICITACAO",
    "NUMEROPROCESSO",
    "CARACTERISTICAOBJETO",
    "CODIGONATUREZA",
    "SITUACAOLICITACAO",
    "DATAPUBLICACAOHABILITACAO",
    "ANOMODALIDADE",
    "ESPECIFICACAOOBJETO",
    "DATAEMISSAOEDITAL",
    "ANOPROCESSO",
    "arquivo",
    "DATASESSAOABERTURA",
    "DOTACAOORCAMENTARIA",
    "FUNDAMENTOLEGAL",
    "RESULTADOHABILITACAO",
    "ADJUDICADA",
    "CODIGOMUNICIPIO",
]

# Dictionary

pe_rename = {
    "CODIGOUG": "id_unidade_gestora",
    "NUMERODOCUMENTOAJUSTADO": "documento",
    "CODIGOPL": "id_licitacao",
    "RAZAOSOCIAL": "razao_social",
}

status = {"Vencedor": "1", "Não Vencedor": "0"}
habilitacao = {
    "Habilitado": "1",
    "Inabilitado": "0",
    "Não analisado": "0",
    "Dispensado": "0",
}

# Drop and rename

pe = pe.rename(pe_rename, axis=1)

pe["vencedor"] = pe["ADJUDICADA"].map(status)
pe["habilitado"] = pe["RESULTADOHABILITACAO"].map(habilitacao)

pe = pe.drop(columns_pe, axis=1)

# Read a CSV file containing municipality information

ug = pd.read_csv(
    os.path.join(path, "input/PE/unidadesjurisdicionadas.csv"),
    sep=",",
    encoding="latin-1",
    dtype=str,
)

ug_drop = [
    "CODIGOTCE",
    "ESFERA",
    "PODER",
    "UNIDADEFEDERATIVA",
    "NATUREZA",
    "TIPOPESSOAJURIDICA",
    "ORGAO",
    "MUNICIPIO",
    "SIGLA",
    "SITUACAO",
    "CNPJ",
]
ug_rename = {
    "CODIGOMUNICIPIO": "id_municipio_tce",
    "ID_UNIDADE_GESTORA": "id_unidade_gestora",
}

ug = ug.drop(ug_drop, axis=1)
ug = ug.rename(columns=ug_rename)

# Merge on id_unidade_gestora to get id_municipio_tce (TCE code)

pe = pe.merge(
    ug,
    how="left",
    left_on=["id_unidade_gestora"],
    right_on=["id_unidade_gestora"],
    indicator=True,
)

pe = pe.drop("_merge", axis=1)

# Merge on id_municipio_tce to get id_municipio (IBGE code)

pe = pe.merge(
    municipio,
    how="left",
    left_on="id_municipio_tce",
    right_on="id_municipio_tce",
)

pe = pe.drop(["nome", "id_municipio_6", "id_municipio_tce"], axis=1)

# Many documents with zeros: 12718 observations (just for non-winners)
# Apparently deserted bids

# Create type variable based on document
# 1 - CNPJ
# 2 - CPF

pe["documento"] = pe["documento"].str.replace("-", "")
pe["documento"] = pe["documento"].str.replace(".", "")
pe["documento"] = pe["documento"].str.strip()

pe["length"] = pe["documento"].str.len()

conditions = [
    (pe["length"] == 14),
    (pe["length"] == 11),
    (pe["length"] != 11) & (pe["length"] != 14),
]
categories = ["1", "2", ""]
pe["tipo"] = np.select(conditions, categories)

pe["tipo"] = pe["tipo"].replace("", np.nan)

# Create a unique identifier for each purchase

pe["id_licitacao_bd"] = (
    pe["id_licitacao"] + pe["id_unidade_gestora"] + pe["sigla_uf"]
)

# Reorder columns

pe = pe.reindex(columns=ordem)

# Save

pe.to_csv(
    os.path.join(path, "output/licitacao_participante_pe.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

# MG


In [ ]:
# List municipalities to loop

municipio_mg = municipio.query('sigla_uf=="MG"')
municipios_mg = municipio_mg["id_municipio"].tolist()

# List years
anos_mg = ["2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021"]

# MG folder
pasta = os.path.join(path, "input/MG")

# Rename and drop variables

mg_rename = {
    "seq_licitacao": "id_licitacao",
    "seq_orgao": "orgao",
    "num_ano_referencia": "ano",
    "num_documento": "documento",
    "nom_pessoa": "razao_social",
}

mg_drop = [
    "seq_hab_licitacao",
    "num_mes_referencia",
    "dsc_objeto_social",
    "dsc_tipo_orgao_resp",
    "dat_registro",
    "num_registro",
    "dat_registro_cvm",
    "num_registro_cvm",
    "num_inscr_estadual",
    "dsc_estado_inscr",
    "num_certidao_inss",
    "dat_emi_cert_inss",
    "dat_val_cert_inss",
    "num_certidao_fgts",
    "dat_emi_cert_fgts",
    "dat_val_cert_fgts",
    "num_cndt",
    "dat_emi_cndt",
    "dat_val_cndt",
    "dat_habilitacao",
    "dsc_ind_presenca_licit",
    "dsc_ind_renuncia",
    "num_versao_arq",
]

# Initialize an empty list and loop through each CSV file
# Qualified participants (Habilitados)

all_df_mg = []
for a in anos_mg:
    for m in municipios_mg:
        path = os.path.join(pasta, f"{a}/licitacao_{a}.zip")
        if a in ["2014", "2015", "2016"]:
            arquivo = f"licitacao/{m}/{a}.{m}.licitacao.habLicitacao.csv"
        elif a == "2017":
            arquivo = f"{a}/licitacao/{m}/{a}.{m}.licitacao.habLicitacao.csv"
        else:
            arquivo = f"{a}.{m}.licitacao.habLicitacao.csv"

        with ZipFile(path) as z:
            try:
                with z.open(arquivo) as f:
                    df = pd.read_csv(f, sep=";", encoding="utf-8", dtype=str)
                    df["id_municipio"] = m
                    df = df.rename(mg_rename, axis=1)
                    df = df.drop(mg_drop, axis=1)
                    df["habilitado"] = "1"
                    df["sigla_uf"] = "MG"
                    all_df_mg.append(df)
            except OSError:
                print(f"Erro de input/output para o município {m} e ano {a}")

mg_habilitados = pd.concat(all_df_mg, ignore_index=True, sort=True)

# Drop duplicates (0.02% only change in seq_hab_licitacao)

mg_habilitados = mg_habilitados.drop_duplicates(
    subset=[
        "documento",
        "razao_social",
        "id_licitacao",
        "id_municipio",
        "ano",
        "orgao",
    ],
)

# Merge to get id_unidade_gestora
# 120 on left_only

mg_habilitados = mg_habilitados.merge(
    ug_id,
    how="left",
    left_on=["ano", "id_municipio", "id_licitacao"],
    right_on=["ano", "id_municipio", "id_licitacao"],
)

# Create a unique identifier for each purchase

mg_habilitados["id_licitacao_bd"] = (
    mg_habilitados["id_licitacao"]
    + mg_habilitados["id_unidade_gestora"]
    + mg_habilitados["sigla_uf"]
)

# We still need to get suppliers

# Open items files and select useful variables
chunksize = 10000
csv_files = [
    os.path.join(path, "output/temp/mg_item_1.csv"),
    os.path.join(path, "output/temp/mg_item_2.csv"),
    os.path.join(path, "output/temp/mg_item_3.csv"),
]

all_df = []

for file in csv_files:
    df_chunk = pd.read_csv(
        file,
        dtype=str,
        encoding="utf-8",
        usecols=[
            "ano",
            "id_municipio",
            "orgao",
            "id_unidade_gestora",
            "id_licitacao_bd",
            "id_licitacao",
            "id_dispensa",
            "nome_vencedor",
            "documento",
        ],
        chunksize=chunksize,
    )
    chunk_list = []
    for chunk in df_chunk:
        chunk_list.append(chunk)
    df = pd.concat(chunk_list)
    all_df.append(df)

licitacao_item_mg = pd.concat(all_df, ignore_index=True, sort=True)

# Keep unique winners

licitacao_item_mg = licitacao_item_mg.drop_duplicates(
    subset=[
        "documento",
        "nome_vencedor",
        "id_licitacao_bd",
        "id_municipio",
        "ano",
        "orgao",
    ],
)

licitacao_item_mg["vencedor"] = "1"

licitacao_item_mg = licitacao_item_mg.rename(
    {"nome_vencedor": "razao_social"}, axis=1
)

# Merge to get suppliers

mg = mg_habilitados.merge(
    licitacao_item_mg,
    how="outer",
    left_on=[
        "ano",
        "id_municipio",
        "id_licitacao_bd",
        "id_licitacao",
        "orgao",
        "id_unidade_gestora",
        "documento",
        "razao_social",
    ],
    right_on=[
        "ano",
        "id_municipio",
        "id_licitacao_bd",
        "id_licitacao",
        "orgao",
        "id_unidade_gestora",
        "documento",
        "razao_social",
    ],
    indicator=True,
)

# both          898133
# right_only    327541
# left_only     126115

mg["vencedor"] = mg["vencedor"].replace(np.nan, "0")

# Create type variable according to document characters
# 1 if cnpj
# 2 if cpf

mg["documento"] = mg["documento"].str.replace("-", "")
mg["documento"] = mg["documento"].str.replace(".", "")
mg["documento"] = mg["documento"].str.strip()

mg["length"] = mg["documento"].str.len()

conditions = [
    (mg["length"] == 14),
    (mg["length"] == 11),
    (mg["length"] != 11) | (mg["length"] != 14),
]
categories = ["1", "2", ""]
mg["tipo"] = np.select(conditions, categories)

mg["tipo"] = mg["tipo"].str.replace("0", "")

# Format
mg["ano"] = mg["ano"].astype(int)

# Assign state acronym to the 'sigla_uf'
mg["sigla_uf"] = "MG"

# Keep only non-missing information
mg = mg[(mg["documento"].notna()) & (mg["razao_social"].notna())]

# Reorder columns
mg = mg.reindex(columns=ordem)

# Save
mg.to_csv(
    os.path.join(path, "output/licitacao_participante_mg.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

# PR


In [ ]:
# PR

# List municipalities to loop

municipio_pr = municipio.query('sigla_uf=="PR"')

municipio_pr = municipio_pr.query(
    'id_municipio_6 != "411915" & \
                                  id_municipio_6 != "411370" & \
                                  id_municipio_6 != "411535" & \
                                  id_municipio_6 != "411710" & \
                                  id_municipio_6 != "412627" & \
                                  id_municipio_6 != "410140" & \
                                  id_municipio_6 != "410350"'
)

municipios_pr = municipio_pr["id_municipio_6"].tolist()

anos_pr = [
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
]

# Initialize an empty list and loop through each CSV file

all_files_pr = []
for a in anos_pr:
    for m in municipios_pr:
        exec(
            f"path_lic_par = '/content/gdrive/MyDrive/ComprasPublicas_Brasil/input/PR/{a}/Licitacao/{m}/{a}_{m}_LicitacaoParticipante.csv'"
        )
        all_files_pr.append(path)

# Identify file origin
all_df_pr = []
for f in all_files_pr:
    df1 = pd.read_csv(f, sep=",", encoding="utf-8", dtype=str)
    df1["arquivo"] = f.split("/")[-1]
    all_df_pr.append(df1)

pr1 = pd.concat(all_df_pr, ignore_index=True, sort=True)

# Drop and rename

pr1_drop = [
    "DataReferencia",
    "arquivo",
    "nmEntidade",
    "nrLicitacao",
    "ultimoEnvioSIMAMNesteExercicio",
    "dtOcorrencia",
    "nmMunicipio",
]

pr1 = pr1.drop(pr1_drop, axis=1)

pr1_rename = {
    "cdIBGE": "id_municipio",
    "idLicitacao": "id_licitacao",
    "idEntidade": "id_unidade_gestora",
    "nrAnoLicitacao": "ano",
    "nmParticipanteLicitacao": "razao_social",
    "nrDocParticipanteLicitacao": "documento",
    "sgDocParticipanteLicitacao": "tipo",
}

pr1 = pr1.rename(pr1_rename, axis=1)

# Format

pr1["tipo"] = pr1["tipo"].str.strip()
pr1["tipo"] = pr1["tipo"].replace(["CNPJ", "CPF"], ["1", "2"])

# Reshape - long to wide

classificados_pr = pr1[
    (pr1["dsTipoSituacaoParticipante"] == "Classificado")
    | (pr1["dsTipoSituacaoParticipante"] == "Desclassificado")
]

classificado = {"Classificado": "1", "Desclassificado": "0"}
classificados_pr["classificado"] = classificados_pr[
    "dsTipoSituacaoParticipante"
].map(classificado)
classificados_pr = classificados_pr.drop("dsTipoSituacaoParticipante", axis=1)

habilitados_pr = pr1[
    (pr1["dsTipoSituacaoParticipante"] == "Habilitado")
    | (pr1["dsTipoSituacaoParticipante"] == "Desabilitado")
]
habilitado = {"Habilitado": "1", "Desabilitado": "0"}
habilitados_pr["habilitado"] = habilitados_pr[
    "dsTipoSituacaoParticipante"
].map(habilitado)
habilitados_pr = habilitados_pr.drop("dsTipoSituacaoParticipante", axis=1)

# Merge classificados and habilitados
pr = classificados_pr.merge(
    habilitados_pr,
    how="outer",
    left_on=[
        "id_licitacao",
        "documento",
        "ano",
        "id_municipio",
        "id_unidade_gestora",
        "razao_social",
        "tipo",
    ],
    right_on=[
        "id_licitacao",
        "documento",
        "ano",
        "id_municipio",
        "id_unidade_gestora",
        "razao_social",
        "tipo",
    ],
)

# Participants level to tender level
# Deal with repetead rows for some participants
pr = pr.drop_duplicates(
    subset=[
        "id_municipio",
        "id_unidade_gestora",
        "id_licitacao",
        "ano",
        "documento",
    ]
)

# Format
pr["documento"] = pr["documento"].str.replace("-", "")
pr["documento"] = pr["documento"].str.replace(".", "")
pr["documento"] = pr["documento"].str.strip()

# Merge id_municipio 6 and 7 digits id

pr["id_municipio"] = pr["id_municipio"].astype("string")

pr = pr.merge(
    municipio,
    how="left",
    left_on="id_municipio",
    right_on="id_municipio_6",
)

pr = pr.drop(["id_municipio_x", "id_municipio_6", "nome"], axis=1)
pr = pr.rename({"id_municipio_y": "id_municipio"}, axis=1)

# Create a unique identifier for each purchase

pr["id_licitacao_bd"] = (
    pr["id_licitacao"] + pr["id_unidade_gestora"] + pr["sigla_uf"]
)

# Get winner information from licitacao_item

item_pr1 = pd.read_csv(
    os.path.join("output/temp/item_pr1.csv"),
    dtype=str,
    encoding="utf-8",
    usecols=[
        "ano",
        "id_municipio",
        "id_licitacao_bd",
        "nome_vencedor",
        "documento",
    ],
)

item_pr2 = pd.read_csv(
    os.path.join("output/temp/item_pr2.csv"),
    dtype=str,
    encoding="utf-8",
    usecols=[
        "ano",
        "id_municipio",
        "id_licitacao_bd",
        "nome_vencedor",
        "documento",
    ],
)

item_pr3 = pd.read_csv(
    os.path.join("output/temp/item_pr3.csv"),
    dtype=str,
    encoding="utf-8",
    usecols=[
        "ano",
        "id_municipio",
        "id_licitacao_bd",
        "nome_vencedor",
        "documento",
    ],
)

item_pr = item_pr1.append([item_pr2, item_pr3], ignore_index=True)

# Make each supplier only appear once

item_pr = item_pr.drop_duplicates(
    subset=[
        "documento",
        "nome_vencedor",
        "id_licitacao_bd",
        "id_municipio",
        "ano",
    ],
)

item_pr["vencedor"] = "1"

item_pr = item_pr.rename({"nome_vencedor": "razao_social"}, axis=1)

# Then Merge
pr = pr.merge(
    item_pr,
    how="left",
    left_on=[
        "ano",
        "id_municipio",
        "id_licitacao_bd",
        "razao_social",
        "documento",
    ],
    right_on=[
        "ano",
        "id_municipio",
        "id_licitacao_bd",
        "razao_social",
        "documento",
    ],
    indicator=True,
)

# Participants who are not in licitacao_item, are not winners
pr["vencedor"] = pr["vencedor"].replace(np.nan, "0")

# Reorder columns
pr = pr.reindex(columns=ordem)

# Save
pr.to_csv(
    os.path.join(path, "output/licitacao_participante_pr.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

# RS


In [ ]:
# List municipalities to loop

municipio_rs = municipio.query('sigla_uf=="RS"')
municipios_rs = municipio_rs["id_municipio"].tolist()

# Rename and Drop variables

rs_rename = {
    "ANO_LICITACAO": "ano",
    "NR_DOCUMENTO": "documento",
    "NR_LICITACAO": "id_licitacao",
    "TP_RESULTADO_HABILITACAO": "habilitado",
    "TP_DOCUMENTO": "tipo",
    "CD_TIPO_MODALIDADE": "modalidade",
}

rs_drop = [
    "BL_BENEFICIO_MICRO_EPP",
    "TP_DOCUMENTO.1",
    "TP_CONDICAO",
    "NR_DOCUMENTO.1",
]

In [ ]:
anos_rs = ["2016", "2017", "2018", "2019", "2020", "2021"]

# RS folder
pasta = os.path.join(path, "input/RS")

all_df_rs = []
for a in anos_rs:
    df = os.path.join(
        pasta,
        f"Licitacao/{a}.csv.zip",
    )
    with ZipFile(df) as z:
        with z.open("licitante.csv") as f:
            rs = pd.read_csv(f, sep=",", encoding="utf-8", dtype=str)

            rs = rs.drop(rs_drop, axis=1)

            # In PRD(Processo de Dispensa), PRI(Processo de Inexigibilidade) and
            # RPO(Adesão à Ata de Registro de Preços) there is no participant registry
            # For other purchase types, there is registry and any missing

            # Replace documents with non-numerical strings (usually suppliers name)
            rs["NR_DOCUMENTO"] = np.where(
                rs["NR_DOCUMENTO"].str.isnumeric(), rs["NR_DOCUMENTO"], np.nan
            )

            # Adds zeros to the left, missing in some cnpjs
            rs["length"] = rs["NR_DOCUMENTO"].str.len()
            rs["NR_DOCUMENTO"] = np.where(
                (rs["length"] > 11) & (rs["length"] < 14),
                rs["NR_DOCUMENTO"].str.zfill(14),
                rs["NR_DOCUMENTO"],
            )
            rs = rs.drop(["length"], axis=1)

        # Files'licitacao' and 'itens' have information about PRD, PRI, RPO

        # licitacao
        with z.open("licitacao.csv") as f:
            rs2 = pd.read_csv(
                f,
                sep=",",
                encoding="utf-8",
                dtype=str,
                usecols=[
                    "CD_ORGAO",
                    "NR_LICITACAO",
                    "ANO_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                    "TP_DOCUMENTO_VENCEDOR",
                    "NR_DOCUMENTO_VENCEDOR",
                    "TP_DOCUMENTO_FORNECEDOR",
                    "NR_DOCUMENTO_FORNECEDOR",
                ],
            )

        # Many documents as missing (~67%)
        # rs2['NR_DOCUMENTO_VENCEDOR'].isna() & rs2['NR_DOCUMENTO_FORNECEDOR'].isna()

        # Items files to get more participant information
        with z.open("item.csv") as f:
            rs3 = pd.read_csv(
                f,
                sep=",",
                encoding="utf-8",
                dtype=str,
                usecols=[
                    "ANO_LICITACAO",
                    "NR_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                    "NR_DOCUMENTO.1",
                    "NR_DOCUMENTO",
                    "CD_ORGAO",
                    "TP_DOCUMENTO.1",
                    "TP_DOCUMENTO",
                ],
            )

            # Two different variables for document, one of them includes information when the purchase type is PRD, PRI or RPO.
            # Concat both
            rs3["NR_DOCUMENTO"] = np.where(
                rs3["NR_DOCUMENTO"].isna(),
                rs3["NR_DOCUMENTO.1"],
                rs3["NR_DOCUMENTO"],
            )
            rs3["TP_DOCUMENTO"] = np.where(
                rs3["TP_DOCUMENTO"].isna(),
                rs3["TP_DOCUMENTO.1"],
                rs3["TP_DOCUMENTO"],
            )

            # Drop non necessary variables
            rs3 = rs3.drop(["NR_DOCUMENTO.1", "TP_DOCUMENTO.1"], axis=1)

            # We may have a supplier winning many items within the same tender
            # Drop duplicated suppliers

            rs3 = rs3.drop_duplicates(
                subset=[
                    "NR_DOCUMENTO",
                    "CD_ORGAO",
                    "ANO_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                    "NR_LICITACAO",
                ],
            )

            # Merge licitacao and items

            rs2 = rs2.merge(
                rs3,
                how="left",
                left_on=[
                    "CD_ORGAO",
                    "NR_LICITACAO",
                    "ANO_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                ],
                right_on=[
                    "CD_ORGAO",
                    "NR_LICITACAO",
                    "ANO_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                ],
            )

            # Concat all document variables
            rs2["NR_DOCUMENTO"] = np.where(
                rs2["NR_DOCUMENTO"].isna()
                & rs2["NR_DOCUMENTO_FORNECEDOR"].notna(),
                rs2["NR_DOCUMENTO_FORNECEDOR"],
                rs2["NR_DOCUMENTO"],
            )
            rs2["NR_DOCUMENTO"] = np.where(
                rs2["NR_DOCUMENTO"].isna()
                & rs2["NR_DOCUMENTO_VENCEDOR"].notna(),
                rs2["NR_DOCUMENTO_VENCEDOR"],
                rs2["NR_DOCUMENTO"],
            )

            rs2["TP_DOCUMENTO"] = np.where(
                rs2["TP_DOCUMENTO"].isna()
                & rs2["TP_DOCUMENTO_FORNECEDOR"].notna(),
                rs2["TP_DOCUMENTO_FORNECEDOR"],
                rs2["TP_DOCUMENTO"],
            )
            rs2["TP_DOCUMENTO"] = np.where(
                rs2["TP_DOCUMENTO"].isna()
                & rs2["TP_DOCUMENTO_VENCEDOR"].notna(),
                rs2["TP_DOCUMENTO_VENCEDOR"],
                rs2["TP_DOCUMENTO"],
            )

            rs2 = rs2.rename({"NR_DOCUMENTO": "NR_DOCUMENTO_2"}, axis=1)
            rs2 = rs2.rename({"TP_DOCUMENTO": "TP_DOCUMENTO_2"}, axis=1)

            # Replace non-numerical documents (usually suppliers name)
            rs2["NR_DOCUMENTO_2"] = np.where(
                rs2["NR_DOCUMENTO_2"].str.isnumeric(),
                rs2["NR_DOCUMENTO_2"],
                np.nan,
            )

            # Adds zeros to the left, missing in some cnpjs
            rs2["length"] = rs2["NR_DOCUMENTO_2"].str.len()
            rs2["NR_DOCUMENTO_2"] = np.where(
                (rs2["length"] > 11) & (rs2["length"] < 14),
                rs2["NR_DOCUMENTO_2"].str.zfill(14),
                rs2["NR_DOCUMENTO_2"],
            )
            rs2 = rs2.drop(["length"], axis=1)

            # Subset with PRD, PRI e RPO to merge outer
            rs2_2 = rs2[
                (rs2["CD_TIPO_MODALIDADE"] == "PRD")
                | (rs2["CD_TIPO_MODALIDADE"] == "PRI")
                | (rs2["CD_TIPO_MODALIDADE"] == "RPO")
            ]

            # Merge main licitante dataframe with the above subset to get participants information for PRD,PRI,RPO
            rs = rs.merge(
                rs2_2,
                how="outer",
                left_on=[
                    "CD_ORGAO",
                    "NR_LICITACAO",
                    "ANO_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                ],
                right_on=[
                    "CD_ORGAO",
                    "NR_LICITACAO",
                    "ANO_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                ],
            )

            # Concat documents to create only one variable
            rs["NR_DOCUMENTO"] = np.where(
                rs["NR_DOCUMENTO"].isna(),
                rs["NR_DOCUMENTO_2"],
                rs["NR_DOCUMENTO"],
            )
            rs["TP_DOCUMENTO"] = np.where(
                rs["TP_DOCUMENTO"].isna(),
                rs["TP_DOCUMENTO_2"],
                rs["TP_DOCUMENTO"],
            )

            # Still ~5000 documents missings, all related to PRD, PRI, RPO.
            # Will drop those observations, as they do not add aditional information here.
            rs = rs.dropna(subset=["NR_DOCUMENTO"])

            rs = rs.drop(
                [
                    "NR_DOCUMENTO_2",
                    "NR_DOCUMENTO_FORNECEDOR",
                    "NR_DOCUMENTO_VENCEDOR",
                    "TP_DOCUMENTO_2",
                    "TP_DOCUMENTO_FORNECEDOR",
                    "TP_DOCUMENTO_VENCEDOR",
                ],
                axis=1,
            )

            # I still need to identify who are the winners
            # Merge rs and rs2 (rs2 comes from licitacao or items, thus it only reports winners)

            rs = rs.merge(
                rs2,
                how="left",
                left_on=[
                    "CD_ORGAO",
                    "NR_LICITACAO",
                    "ANO_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                    "NR_DOCUMENTO",
                ],
                right_on=[
                    "CD_ORGAO",
                    "NR_LICITACAO",
                    "ANO_LICITACAO",
                    "CD_TIPO_MODALIDADE",
                    "NR_DOCUMENTO_2",
                ],
            )

            rs["vencedor"] = np.where(
                rs["NR_DOCUMENTO"] == rs["NR_DOCUMENTO_2"], 1, 0
            )

            # Drop non necessary variables
            rs = rs.drop(
                [
                    "NR_DOCUMENTO_2",
                    "NR_DOCUMENTO_FORNECEDOR",
                    "NR_DOCUMENTO_VENCEDOR",
                    "TP_DOCUMENTO_2",
                    "TP_DOCUMENTO_FORNECEDOR",
                    "TP_DOCUMENTO_VENCEDOR",
                ],
                axis=1,
            )

            # Rename
            rs = rs.rename(rs_rename, axis=1)

        # Open file with information about corporate name (razao_social) and other information
        with z.open("pessoas.csv") as f:
            rs4 = pd.read_csv(
                f,
                sep=",",
                encoding="utf-8",
                dtype=str,
                usecols=[
                    "NR_DOCUMENTO",
                    "NM_PESSOA",
                    "CEP",
                    "TP_PESSOA",
                    "LOGRADOURO",
                    "NR_ENDERECO",
                ],
            )

            rs4 = rs4.rename(
                {
                    "NM_PESSOA": "razao_social",
                    "NR_DOCUMENTO": "documento",
                    "CEP": "cep",
                },
                axis=1,
            )

            rs4["endereco"] = rs4["LOGRADOURO"] + "," + rs4["NR_ENDERECO"]

            # Format
            rs4["cep"] = rs4["cep"].astype(float)
            rs4["cep"] = rs4["cep"].apply(lambda x: f"{x:.0f}")
            rs4["cep"] = rs4["cep"].astype(str)
            rs4["cep"] = rs4["cep"].replace("nan", np.nan)

            rs4 = rs4.drop(["LOGRADOURO", "NR_ENDERECO"], axis=1)

            # Replace documents with non-numerical strings (usually suppliers name)
            rs4["documento"] = np.where(
                rs4["documento"].str.isnumeric(), rs4["documento"], np.nan
            )

            # Adds zeros to the left, missing in some cnpjs
            rs4["length"] = rs4["documento"].str.len()
            rs4["documento"] = np.where(
                (rs4["length"] > 11) & (rs4["length"] < 14),
                rs4["documento"].str.zfill(14),
                rs4["documento"],
            )
            rs4 = rs4.drop(["length"], axis=1)

            # Drop duplicates
            rs4 = rs4.drop_duplicates(subset=["documento"])
            rs4 = rs4.dropna(subset=["documento"])

            # Merge to get suppliers name
            rs = rs.merge(
                rs4,
                how="left",
                left_on=["documento"],
                right_on=["documento"],
            )

            # Drop MAI(Manifestação de Interesse)
            rs = rs.drop(rs[(rs["modalidade"] == "MAI")].index)

            # Recode some variables
            rs["habilitado"] = rs["habilitado"].replace(
                ["H", "I", "N"], ["1", "0", ""]
            )  # N - Did not attend, exclusive to the invitation modality
            rs["tipo"] = rs["tipo"].replace(
                ["J", "F", "E", "P"], ["1", "2", "3", ""]
            )
            rs["tipo"] = np.where(
                (rs["documento"] == "05996565000194") & (rs["tipo"] == "3"),
                "1",
                rs["tipo"],
            )

            rs["sigla_uf"] = "RS"

            # Create a unique identifier for each purchase
            rs["id_licitacao_bd"] = (
                rs["id_licitacao"]
                + rs["ano"]
                + rs["modalidade"]
                + rs["CD_ORGAO"]
                + rs["sigla_uf"]
            )

            # Merge to get id_municipio

            rs = rs.merge(
                orgao_municipio,
                how="left",
                left_on="CD_ORGAO",
                right_on="CD_ORGAO",
                indicator=True,
            )

            rs = rs.drop(
                [
                    "NOME_ORGAO",
                    "ESFERA",
                    "SIGLA_ORGAO",
                    "SETOR_GOVERNAMENTAL",
                    "CNPJ",
                    "HOME_PAGE",
                    "NATUREZA_JURIDICA",
                    "CONTABILIDADE",
                    "SITUACAO_ORGAO",
                    "CD_MUNICIPIO_TCERS",
                    "NOME_MUNICIPIO",
                ],
                axis=1,
            )

            rs = rs.rename(
                {"CD_MUNICIPIO_IBGE": "id_municipio", "CD_ORGAO": "orgao"},
                axis=1,
            )

            rs = rs[rs["_merge"] == "both"]

            rs = rs.drop("_merge", axis=1)

    all_df_rs.append(rs)

rs = pd.concat(all_df_rs, ignore_index=True, sort=True)

# Drop duplicates in all rows (0.01% obs)
rs = rs.drop_duplicates()

# Reorder columns
rs = rs.reindex(columns=ordem)

# Save
rs.to_csv(
    os.path.join(path, "output/licitacao_rs.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

# PB


In [ ]:
# PB

# Open files

pb = pd.read_csv(
    os.path.join(
        path, "input/PB/TCE-PB-Portal-Gestor-Licitacoes_Propostas.txt"
    ),
    sep="|",
    encoding="utf-8",
    dtype=str,
)

pb_drop = [
    "jurisdicionado_id",
    "nome_jurisdicionado",
    "nome_tipo_jurisdicionado",
    "nome_tipo_administracao_jurisdicionado",
    "nome_esfera_jurisdicionado",
    "nome_modalidade_licitacao",
    "nome_setor_atual_licitacao",
    "url",
    "objeto_licitacao",
    "valor_estimado_licitacao",
    "valor_proposta",
    "situacao_proposta",
    "valor_licitado_licitacao",
    "nome_estagio_processual_licitacao",
    "situacao_fracassada_licitacao",
    "situacao_proposta",
    "data_homologacao_licitacao",
]

pb_rename = {
    "ano_homologacao_licitacao": "ano",
    "cd_ugestora": "id_unidade_gestora",
    "cpf_cnpj_proponente": "documento",
    "nome_proponente": "razao_social",
    "protocolo_licitacao": "id_licitacao",
}

pb = pb.rename(pb_rename, axis=1)

# print(pb['nome_municipio'].isna().sum())
# 50298 missing values, consórcios intermunicipais regionais: ex - Consórcio Intermunicipal de Saúde dos Municípios do Alto Sertão Paraibano
# or state level

pb = pb.dropna(subset=["nome_municipio"])

# Extract year from numero_licitacao
pb["ano"] = pb["numero_licitacao"].str[6:10]
pb["ano"] = pb["ano"].str.replace("2104", "2014")
pb["ano"] = pb["ano"].str.replace("3014", "2014")

pb["sigla_uf"] = "PB"

# Assign winner
pb["vencedor"] = np.where(pb["situacao_proposta"] == "Vencedora", "1", "0")

# Drop non-necessary variables
pb = pb.drop(pb_drop, axis=1)

# Merge to get id_municipio
pb["nome_municipio"] = pb["nome_municipio"].str.replace(
    "Santa Terezinha", "Santa Teresinha"
)
pb["nome_municipio"] = pb["nome_municipio"].str.replace("Quixaba", "Quixabá")

pb = pb.merge(
    municipio,
    how="left",
    left_on=["nome_municipio", "sigla_uf"],
    right_on=["nome", "sigla_uf"],
    indicator=True,
)
pb = pb.drop(
    ["nome", "nome_municipio", "id_municipio_6", "id_municipio_tce"],
    axis=1,
)

# Format
pb["id_licitacao"] = pb["id_licitacao"].str[5:]
pb["id_licitacao"] = pb["id_licitacao"].str.replace("/", "")
pb["ano"] = pb["ano"].astype(int)

# Create a unique identifier for each purchase
pb["id_licitacao_bd"] = (
    pb["id_licitacao"] + pb["id_unidade_gestora"] + pb["sigla_uf"]
)

# Assign participant type (1- CNPJ, 2- CPF)
pb["length"] = pb["documento"].str.len()
pb["tipo"] = np.where((pb["length"] == 14), "1", "2")

# Drop years after 2021
pb["ano"] = pb["ano"].astype(int)
pb = pb[pb["ano"] < 2022]

# Duplicates drop (0.11%, only changes on proposal value)
pb = pb.drop_duplicates(["id_licitacao_bd", "razao_social", "documento"])

# Reorder columns
pb = pb.reindex(columns=ordem)

# Save
pb.to_csv(
    os.path.join(path, "output/licitacao_participante_pb.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

# Partition


In [ ]:
# List of UFs
ufs = ["CE", "PE", "MG", "PR", "RS", "PB"]

# Loop over each UF
for uf in ufs:
    # Load the corresponding CSV file for the UF
    file_path = f"/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao_participante_{uf.lower()}.csv"
    df = pd.read_csv(file_path, dtype=str, encoding="utf-8")

    # Convert 'ano' column to integer
    df["ano"] = df["ano"].astype(int)

    # Save cvs by year and state

    for ano in [*range(2012, 2022)]:
        for uf in ufs:
            if uf == "CE" and ano in [*range(2009, 2022)]:
                print(f"Particionando {ano} do CE")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao_participante/ano={ano}/sigla_uf=CE/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "PE" and ano in [*range(2012, 2022)]:
                print(f"Particionando {ano} do PE")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao_participante/ano={ano}/sigla_uf=PE/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "MG" and ano in [*range(2014, 2022)]:
                print(f"Particionando {ano} do MG")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao_participante/ano={ano}/sigla_uf=MG/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "PR" and ano in [*range(2013, 2022)]:
                print(f"Particionando {ano} do PR")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao_participante/ano={ano}/sigla_uf=PR/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "RS" and ano in [*range(2016, 2022)]:
                print(f"Particionando {ano} do RS")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao_participante/ano={ano}/sigla_uf=RS/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "PB" and ano in [*range(2014, 2022)]:
                print(f"Particionando {ano} do PB")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao_participante/ano={ano}/sigla_uf=PB/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )